In [1]:
import getpass
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

os.environ["LANGCHAIN_TRACING_V2"] = "true"

load_dotenv()
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangChain API key: ")


In [5]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:
    user_name = config["configurable"].get["user_name"]
    system_msg = f"You are a helpful assistant for {user_name}. Address them by their name, as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"] 

# AnyMessage is any message in the chat, whether user or system
# AgentState is a dictionary that keeps track of the current state of the
# its "messages" key contains a list of messages so far
# why is the return constructed this way?

In [14]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool
from pydantic import BaseModel

model = init_chat_model(
    "openai:gpt-4o",
    temperature=0.7
)

class WeatherResponse(BaseModel):
    city: str 
    condition: str 


@tool
def get_weather_tool(city: str) -> str:
    """Returns a fake weather report for the given city."""
    return f"It's always sunny in {city}!"

checkpointer = InMemorySaver()

agent = create_react_agent(
    model = "openai:gpt-4o",
    tools = [get_weather_tool],
    # checkpointer = checkpointer,
    response_format= WeatherResponse,
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
# ny_response = agent.invoke(
#     {"messages": [{"role": "user", "content": "what about new york?"}]},
#     config
# )
print("Response for SF:", sf_response)
print("Structured response for SF:", sf_response["structured_response"].model_dump())

# print("Response for NY:", ny_response)

Response for SF: {'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='3560eeb0-f16c-4b73-9e7c-b2b9aadf8ba1'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SpRRXzPSJ8OskyWYo8v4nt0b', 'function': {'arguments': '{"city":"San Francisco"}', 'name': 'get_weather_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 54, 'total_tokens': 70, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'id': 'chatcmpl-Bmc7yYHLcPHYlVtYK4BGkOeLjEezr', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3f39636f-99e1-49ff-be78-41496f37299e-0', tool_calls=[{'name': 'get_weather_tool', 'args': {'city'